In [540]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff

## 3. Cash flow

In [556]:
def LinearProgrammingExample():
    solver = pywraplp.Solver.CreateSolver('GLOP')
    if not solver:
        return

    x1 = solver.NumVar(0, solver.infinity(), 'x1')
    x2 = solver.NumVar(0, solver.infinity(), 'x2')
    y1 = solver.NumVar(0, solver.infinity(), 'y1')
    y2= solver.NumVar(0, solver.infinity(), 'y2')
    y3 = solver.NumVar(0, solver.infinity(), 'y3')
    y4 = solver.NumVar(0, solver.infinity(), 'y4')
    y5 = solver.NumVar(0, solver.infinity(), 'y5')


    z1=solver.NumVar(0, solver.infinity(), 'z1')
    z2=solver.NumVar(0, solver.infinity(), 'z2')
    z3=solver.NumVar(0, solver.infinity(), 'z3')
    z4=solver.NumVar(0, solver.infinity(), 'z4')
    z5=solver.NumVar(0, solver.infinity(), 'z5')
    z6=solver.NumVar(0, solver.infinity(), 'z6')



    solver.Add(x1 + y1-z1 == 200)

    solver.Add((-0.0125*x1)+x2+y2-(1.02)*y1+z1-z2==400)

    solver.Add((-0.0125*x2)+y3-(1.02)*y2+z2-z3==-100)

    solver.Add(-0.0125*(x1+x2)-1.02*y3+z3-z4 == -100)

    solver.Add(-0.0125*(x1+x2)-1.02*y4+z4-z5==0)

    solver.Add(-1.0125*(x1+x2)-1.02*y5+z5-z6==-550)

    solver.Maximize(z6)

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        print('Objective value =', '$',1000*solver.Objective().Value())
        print('x1 =', 1000*x1.solution_value())
        print('x2 =', 1000*x2.solution_value())
        print('y1 =', 1000*y1.solution_value())
        print('y2 =', 1000*y2.solution_value())
        print('y3 =', 1000*y3.solution_value())
        print('y4 =', 1000*y4.solution_value())
        print('y5 =', 1000*y5.solution_value())
        
    else:
        print('No optimal solution.')
LinearProgrammingExample()

Solution:
Objective value = $ 123560.42736747622
x1 = 200000.0
x2 = 221174.8865506408
y1 = 0.0
y2 = 181325.11344935917
y3 = 87716.30180022937
y4 = 0.0
y5 = 0.0


## Efficient Frontier

### min σP2 = σ12x21 + σ2x2 + σ32x23 + 2(σ12x1x2 + σ23x2x3 + σ13x1x3)
### s.t. μ1x1 + μ2x2 + μ3x3 ≥ μ~
### x1 + x2 + x3 = 1
#### *μP =μ1x1+μ2x2+μ3x3
###  * the target return μ~ should not be below the sector 3 average return (0.1) because otherwise the portfolio should be composed of 100% sector 3
### *the target return won't go above the sector 1 average return (0.16) because there is no way to make a linear combination of portfolios with > 0.16 returns such that the weights add to 1


In [543]:
mus_data={'sector 1':0.16,'sector 2':0.11,'sector 3':0.1}

mus = pd.Series(mus_data)

var_data={'sector 1':0.22,'sector 2':0.1,'sector 3':0.14}
var = pd.Series(var_data)

cov_data={'sector 1':[1,0.05,0.03],'sector 2':[0.05,1,0.02],'sector 3':[0.03,0.02,1]}
cov=pd.DataFrame(cov_data,index=['sector 1','sector 2','sector 3'])

In [544]:
#Generate n portfolios with a variety of randomly sampled weights that add to 1
n_assets = 3
n_portfolios =1000
sharpe=[]
sharpe_5mil=[]
mean_variance_pairs = []
weights_=[]
np.random.seed(75)
assets = np.array(['sector 1','sector 2','sector 3'])
w=[]
w5=[]
for i in range(n_portfolios):

    weights = np.random.dirichlet(np.ones(3))
    portfolio_E_Return = weights[0] * mus.loc[assets[0]]+weights[1] * mus.loc[assets[1]]+weights[2] * mus.loc[assets[2]]
    portfolio_E_Variance=0.5*np.sqrt(((weights[0]**2) * var.loc[assets[0]])+
                                  ((weights[1]**2) * var.loc[assets[1]])+
                                  ((weights[2]**2) * var.loc[assets[2]])+
                                  (2*weights[0]*weights[1]*cov.loc[assets[0],assets[1]]+ 
                                     2*weights[1]*weights[2]*cov.loc[assets[1],assets[2]]+ 
                                     2*weights[0]*weights[2]*cov.loc[assets[0],assets[2]]))

    mean_variance_pairs.append([portfolio_E_Return,portfolio_E_Variance])
   

    w.append(weights)
    

mean_variance_pairs = np.array(mean_variance_pairs)

mean_variance_pairs_5mil=[]



# Generate n portfolios such that the weights add to one and each weights has to be >= 1/6 (5/30 -> has at least 5 mil in each sector)
for i in range(n_portfolios):
    
    y=np.array([1/6,1/6,1/6])
    x=np.random.dirichlet(np.ones(3))*.5
    weights=(y+x)

  
    
    portfolio_E_Return = weights[0] * mus.loc[assets[0]]+weights[1] * mus.loc[assets[1]]+weights[2] * mus.loc[assets[2]]
    portfolio_E_Variance=0.5*np.sqrt(((weights[0]**2) * var.loc[assets[0]])+
                                  ((weights[1]**2) * var.loc[assets[1]])+
                                  ((weights[2]**2) * var.loc[assets[2]])+
                                  (2*weights[0]*weights[1]*cov.loc[assets[0],assets[1]]+ 
                                     2*weights[1]*weights[2]*cov.loc[assets[1],assets[2]]+ 
                                     2*weights[0]*weights[2]*cov.loc[assets[0],assets[2]]))

    mean_variance_pairs_5mil.append([portfolio_E_Return,portfolio_E_Variance])
    w5.append(weights)

mean_variance_pairs_5mil = np.array(mean_variance_pairs_5mil)





In [548]:
#Find portfolio with max sharpe (blue dot)
sharpe=[]
for i in range(0, len(mean_variance_pairs)):
    sharpe.append((mean_variance_pairs[i][0]-risk_free_rate)/(mean_variance_pairs[i][1]))
mean_variance_pairs[np.argmax(sharpe)]

array([0.12092874, 0.13155913])

In [554]:
risk_free_rate=0 #-- Include risk free rate here


fig = go.Figure()
fig.add_trace(go.Scatter(x=mean_variance_pairs[:,1], y=mean_variance_pairs[:,0], 
                      marker=dict(color=(mean_variance_pairs[:,0]-risk_free_rate)/(mean_variance_pairs[:,1]), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale='emrld',
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers'))
fig.add_trace(go.Scatter(x=[mean_variance_pairs[(np.argmax(sharpe))][1]],
                 y=[mean_variance_pairs[(np.argmax(sharpe))][0]],
             mode="markers",marker_color="lightskyblue",marker_size=15))


fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualized Portfolio Risk (Variance)'),
                  yaxis=dict(title='Annualized Portfolio Return'),
                  title='Random Sampled Portfolio Combinations and Efficient Frontier',
                  width=850,
                  height=500)

fig.update_layout(coloraxis_colorbar=dict(title="Sharpe Ratio"))

In [550]:
#portfolio with max sharp with the 5 mil constraint
sharpe_5mil=[]
for i in range(0, len(mean_variance_pairs_5mil)):
    sharpe_5mil.append((mean_variance_pairs_5mil[i][0]-risk_free_rate)/(mean_variance_pairs_5mil[i][1]))

In [558]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=mean_variance_pairs_5mil[:,1], y=mean_variance_pairs_5mil[:,0], 
                      marker=dict(color=(mean_variance_pairs_5mil[:,0]-risk_free_rate)/(mean_variance_pairs_5mil[:,1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale='emrld',
                
                                 ), 
                      mode='markers'))

fig.add_trace(go.Scatter(x=[mean_variance_pairs_5mil[(np.argmax(sharpe_5mil))][1]],
                 y=[mean_variance_pairs_5mil[(np.argmax(sharpe_5mil))][0]],
             mode="markers",marker_color="lightskyblue",marker_size=15))

fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualized Portfolio Risk (Variance)'),
                  yaxis=dict(title='Annualized Portfolio Return'),
                  title='Random Sampled Portfolio Combinations and Efficient Frontier 5 mil Constraint',
                  width=850,
                  height=500)

fig.update_layout(coloraxis_colorbar=dict(title="Sharpe Ratio"))

In [552]:
w[np.argmax(sharpe)]

array([0.27473055, 0.44449057, 0.28077889])

In [553]:
w5[np.argmax(sharpe_5mil)]

array([0.26869712, 0.43650643, 0.29479645])

## The weights of each sector for the max sharpe portfolio are slightly different if we require at least 5 mil in each sector. Additionally, the added constraint effectively slices the lower portion of the frontier 
## The lowest average portfolio return for unspecified sector weights is .1, but the lowest average portfolio return for the 5 mil in each sector case is around 0.11. This makes sense because the only way to achieve a portfolio average return of .1 would be to have 100% of the funds in sector 3.